In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
select_numeric = df.select_dtypes("number")
select_numeric_test = test.select_dtypes("number")

In [5]:
def remove_outliers(df_no_outliers):
    for col in df_no_outliers.columns:
       if col not in ["Id", "SalePrice"]:
        Q1 = df_no_outliers[col].quantile(0.25)
        Q3 = df_no_outliers[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df_no_outliers = df_no_outliers[(df_no_outliers[col] >= lower_bound) & (df_no_outliers[col] <= upper_bound)]
    return df_no_outliers
#remove outliers
cleaned_df = remove_outliers(select_numeric)

In [6]:
X=cleaned_df.reset_index(drop=True)
y = X["SalePrice"]
X = X.drop("SalePrice", axis=1)
X.drop("Id", inplace=True, axis=1)
id = select_numeric_test["Id"]
select_numeric_test = select_numeric_test.drop("Id", axis=1)

In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(X)
X = pd.DataFrame(imputed_data, columns=X.columns)
imputed_data_test = imputer.fit_transform(select_numeric_test)
X_test = pd.DataFrame(imputed_data_test, columns=select_numeric_test.columns)

In [8]:
import statsmodels.api as sm
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()
p_values = model.pvalues
coefficients = model.params
results = pd.DataFrame({'Coefficient': coefficients, 'P-value': p_values})
results_sorted = results.sort_values(by='P-value', ascending=False)
print(results_sorted)

                Coefficient       P-value
GarageYrBlt   -3.480925e+01  8.416372e-01
BsmtFullBath   8.612814e+02  7.909441e-01
TotRmsAbvGrd   7.569618e+02  6.460484e-01
MoSold         2.124163e+02  6.059513e-01
YrSold         5.176281e+02  5.225478e-01
LotFrontage    7.130791e+01  4.531317e-01
1stFlrSF       5.420345e+00  4.452414e-01
WoodDeckSF     9.252298e+00  4.337320e-01
MasVnrArea    -1.068968e+01  3.651173e-01
OpenPorchSF    2.653389e+01  3.273948e-01
MSSubClass    -5.721854e+01  2.276292e-01
FullBath      -5.158776e+03  2.130439e-01
BsmtHalfBath   8.953390e-09  1.756686e-01
KitchenAbvGr  -2.220484e+06  1.756024e-01
LowQualFinSF  -8.320436e-08  1.755573e-01
BsmtFinSF2    -4.704031e-07  1.755157e-01
BsmtUnfSF      5.717176e+00  1.387763e-01
HalfBath      -6.301316e+03  1.193026e-01
Fireplaces     3.888897e+03  9.319374e-02
YearRemodAdd   2.078734e+02  4.193302e-02
GarageArea     2.856513e+01  3.332037e-02
GarageCars     8.390205e+03  3.082289e-02
YearBuilt      3.749430e+02  1.777

In [9]:
# statistically significant variables
X = X[["YearRemodAdd",
"GarageArea",
"GarageCars",
"YearBuilt",
"LotArea",
"OverallCond",
"BedroomAbvGr",
"TotalBsmtSF",
"GrLivArea",
"2ndFlrSF",
"BsmtFinSF1",
"OverallQual"]]
X_test_file = select_numeric_test[["YearRemodAdd",
"GarageArea",
"GarageCars",
"YearBuilt",
"LotArea",
"OverallCond",
"BedroomAbvGr",
"TotalBsmtSF",
"GrLivArea",
"2ndFlrSF",
"BsmtFinSF1",
"OverallQual"]]

In [10]:
import statsmodels.api as sm
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()

In [11]:
f_iloc = 0
with open("submission.csv", 'w') as file:
  for i in id:
    row = X_test_file.iloc[[f_iloc]]
    row_with_const = sm.add_constant(row, has_constant='add')
    a = model.predict(row_with_const)
    if a.item() >0:
      to_write = f"{i},{a.item()}\n"
    else:
      to_write = f"{i},{8000}\n"
    file.write(to_write)
    f_iloc += 1
